# Setup

In [1]:
import os 
os.chdir('/home/imendoza/bliss')

In [2]:
%load_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
import sklearn
import torch

from bliss.datasets import sdss
from bliss.report import add_extra_coadd_info

from astropy.table import Table

In [5]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import plotly.express as px
import plotly.graph_objects as go

In [6]:
# where is the sdss data? 
sdss_dir = '/home/imendoza/bliss/data/sdss/'

In [7]:
# check GPU is configured correctly
device = torch.device('cuda:0')
!echo $CUDA_VISIBLE_DEVICES

5


# Load data

In [8]:
pixel_scale=0.396
run = 94
camcol = 1
field = 12
bands = (2,)
sdss_data = sdss.SloanDigitalSkySurvey(
    sdss_dir=sdss_dir,
    run=run,
    camcol=camcol,
    fields=(field,),
    bands=bands,
    overwrite_cache=True,
    overwrite_fits_cache=True,
)

wcs = sdss_data[0]['wcs'][0]
nelec_per_nmgy = sdss_data[0]['nelec_per_nmgy'].mean()
scene = sdss_data[0]["image"][0]
h,w = scene.shape[-2], scene.shape[-1]
print(h, w)

In [9]:
# load coadd catalog
coadd_catalog_file = 'data/coadd_catalog_94_1_12.fits'
psf_image_file = 'data/psField-000094-1-0012-PSF-image.npy'

add_extra_coadd_info(coadd_catalog_file, psf_image_file, pixel_scale, wcs)

coadd_cat = Table.read(coadd_catalog_file)

# Load models

In [10]:
from bliss.models.binary import BinaryEncoder
from bliss.models.galaxy_encoder import GalaxyEncoder
from bliss.sleep import SleepPhase

In [11]:
# prepare BLISS model 
sleep_file = '/home/imendoza/bliss/models/sdss_sleep.ckpt'
galaxy_encoder_file = '/home/imendoza/bliss/models/sdss_galaxy_encoder.ckpt'
binary_file = '/home/imendoza/bliss/models/sdss_binary.ckpt'


# load models
sleep_net = SleepPhase.load_from_checkpoint(sleep_file).to('cpu')
image_encoder= sleep_net.image_encoder.to(device).eval()
binary_encoder = BinaryEncoder.load_from_checkpoint(binary_file).to(device).eval()
galaxy_encoder = GalaxyEncoder.load_from_checkpoint(galaxy_encoder_file).to(device).eval()
galaxy_decoder = sleep_net.image_decoder.galaxy_tile_decoder.galaxy_decoder.eval().to(device)

# Results

In [19]:
from bliss.predict import predict_on_scene
from bliss.report import coadd_metrics

In [20]:
torch.cuda.empty_cache()

In [22]:
# predict using models 

clen = 300
_scene = torch.from_numpy(scene).reshape(1, 1, h, w)
_, est_params = predict_on_scene(clen, _scene, image_encoder, binary_encoder, 
                                 galaxy_encoder, galaxy_decoder, device) 

  0%|                                                                                                                                                                               | 0/15 [00:00<?, ?it/s]

torch.Size([1, 1, 348, 348])


  7%|███████████▏                                                                                                                                                           | 1/15 [00:01<00:20,  1.48s/it]

torch.Size([1, 1, 348, 348])


 13%|██████████████████████▎                                                                                                                                                | 2/15 [00:02<00:19,  1.47s/it]

...

 20%|█████████████████████████████████▍                                                                                                                                     | 3/15 [00:04<00:17,  1.45s/it]

torch.Size([1, 1, 348, 348])


 27%|████████████████████████████████████████████▌                                                                                                                          | 4/15 [00:05<00:16,  1.45s/it]

torch.Size([1, 1, 289, 348])


AssertionError: 

In [15]:
bp = image_encoder.border_padding
coadd_metrics(coadd_cat, est_params, bp, h, w, clen, mag_cut=25.0, slack=1.0)



'cuda'